## Motivation
Recently I wanted to train an **object detection** model using **TensorFlow** on a **GPU**. The installation that had worked for me about 5-6 months ago didn't seem to work this time. The TF2 OD API has been updated (to version 2.5) since then and the CUDA (v11.2) doesn't seem to be compatible with that offered by Kaggle (v11.0). After much exploration and deliberation I decided to create <a href="https://www.kaggle.com/questions-and-answers/253611">this topic</a>. Eventually I found a rather simple way around it - using an older version of the API. 

This notebook demonstrates installation of (a specific version of) the **TensorFlow 2 Object Detection API**.

### GPU Support for TensorFlow Object Detection API
#### At the time of writing (July 2021), the API uses TF 2.5. Currently, for TensorFlow to run on a (CUDA compatible) GPU the following are required to be present:
#### 1. Nvidia GPU (GTX 650 or newer) 
#### 2. CUDA Toolkit v11.2
#### 3. CuDNN 8.1.0

Source: <a href="https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html">TensorFlow 2 Object Detection API</a> and <a href="https://www.tensorflow.org/install/gpu">TensorFlow</a>

For **STEP 1** and **STEP 2** see NVIDIA System Management Interface. It provides monitoring and management capabilities for NVIDIA's Tesla devices. 

In [ ]:
# Check NVIDIA System Management Interface
!nvidia-smi

1. Let's look <a href=https://en.wikipedia.org/wiki/CUDA> here </a> to see if we have the right GPU. Clearly, Tesla P100-PCIE-16GB **>** GTX 650. **STEP 1** ✔️

2. CUDA 11.0 < 11.2. **STEP 2**  ❌ 

In [ ]:
# If you'd like to see your GPU (maybe to see if it's CUDA compatible)
!lspci | grep -i nvidia

In [ ]:
# If you's like to see the location of CUDA
!whereis cuda

In [ ]:
# If you'd like to see CUDA version
!nvcc --version

For **STEP 3** let's look for an existing installation and then check version

In [ ]:
# Check presence of CuDNN 
!whereis cudnn

In [ ]:
# Check version of cudnn
!cat /usr/include/cudnn_version.h

We see *CUDNN_MAJOR 8* and *CUDNN_MINOR 0* which means version 8.0 < 8.1. **STEP 3** ❌ 

Notably, this does not stop us from using GPU with the TensorFlow API itself.

In [ ]:
# Check is TensorFlow can access GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
# Alternatively, check device name
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# Check for CUDA
tf.test.is_built_with_cuda()

Clearly, TensorFlow is able access GPU and everything seems normal. What we need to ensure is that the **TensorFlow 2 Object Detection API** is able to access the **GPU**.

**What options do we have?**
1. We could upgrade to CUDA v11.2. I found this <a href="https://medium.com/analytics-vidhya/installing-any-version-of-cuda-on-ubuntu-and-using-tensorflow-and-torch-on-gpu-b1a954500786">blog</a> useful for doing so. But it involves installing 2-3GB CUDA Toolkit from <a href="https://developer.nvidia.com/cuda-11.2.0-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=runfilelocal">here</a> and other complicated software engineering that's especially difficult on Kaggle.
2. We can work with an older version of TensorFlow.

**How do we work with an older version?**
1. Install the latest TF2 OD API (v2.5). You'll also find a directory structure with everything you need.
2. Uninstall tf-models-official v2.5
3. Install tensorflow v2.4
4. Install tf-models-official v2.4 (it is compatible with the hardware/software configuration)

### TensorFlow Object Detection API Installation
#### 1. Downloading the TensorFlow Model Garden
>workspace/

>└─ models/

>        ├─ community/

>        ├─ official/

>        ├─ orbit/

>        ├─ research/

>        └── ...
#### 2. Protobuf Installation/Compilation
#### 3. COCO API installation
#### 4. Install the Object Detection API

In [ ]:
# Download models
!git clone --depth 1 https://github.com/tensorflow/models

# Compile proto files 
%cd models/research
!pip install protobuf
!protoc object_detection/protos/*.proto --python_out=.
%cd ..
%cd ..

# Install cocoapi
!pip install cython 
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ..
%cd ..
!cp -r cocoapi/PythonAPI/pycocotools models/research/

# Install object detection api
%cd models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ..
%cd ..

In [ ]:
# View tensorflow installation
!pip freeze | grep tensorflow

In [ ]:
# Uninstall version 2.5
!pip uninstall tf-models-official --yes
# Install version 2.4
!pip install tensorflow==2.4
!pip install tf-models-official==2.4

In [ ]:
# View tensorflow installation
!pip freeze | grep tensorflow

In [ ]:
# Verify TensorFlow 2 Object Detection API installation (optional)
%cd models/research
!python object_detection/builders/model_builder_tf2_test.py
%cd ..
%cd ..

All necessary libraries have been installed, as evident from the success messages at the beginning. The API uses GPU for training.

I'm Akshit Bhalla. I work on machine learning technologies as a Software Engineer. I'm passionate about Deep Learning and I'm a TensorFlow fan. Connect with me on <a href="https://www.linkedin.com/in/akshit-bhalla/">LinkedIn</a>. 

**If you found this notebook helpful or it taught you something new, please consider upvoting it. Thank you!**